In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field
from langchain.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper
from langchain.tools import WikipediaQueryRun, DuckDuckGoSearchRun

llm = ChatOpenAI(
    temperature=0.1,
    model = "gpt-4o-mini")

class SearchUrlToolArgsSchema(BaseModel):
    query: str = Field(description="The query to find website url.Example query: Research about Steve Jobs")


#OPENAI FUNCTION
class DuckDuckGoUrlSearchTool(BaseTool):
    name = "DuckDuckGoUrlSearchTool"
    description = """
    Use this tool to find a **website URL link**. Return the URL link of the most relevant website.
    It takes a query as an argument. If you can't find any relevant website, use WikipediaSearchTool to continue searching.

    Example output: https:/example.com
    """
    args_schema: Type[SearchUrlToolArgsSchema] = SearchUrlToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchRun(api_wrapper=DuckDuckGoSearchAPIWrapper())
        return ddg.run(query)

class WikipediaUrlSearchTool(BaseTool):
    name= "WikipediaUrlSearchTool"
    description="""
    Use this tool to find a **website URL link**. Return the URL link of the most relevant website.
    It takes a query as an argument. If you can't find any relevant website, use DuckDuckGoSearchTool to continue searching.

    Example output: https:/example.com
    """

    args_schema: Type[SearchUrlToolArgsSchema] = SearchUrlToolArgsSchema

    def _run(self, query):
        wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wikipedia.run(query)


In [11]:
from langchain_community.document_loaders import WebBaseLoader

class ExtractWebsiteContextToolArgsSchema(BaseModel):
    url: str = Field(description="website url found by Wikipedia or DuckDuckGo.")


#OPENAI FUNCTION
class ExtractWebsiteContextTool(BaseTool):
    name = "ExtractWebsiteContextTool"
    description = """
    Use url found by Wikipedia or DuckDuckGo to extract and load text from the given url.
    It takes an url as an argument.

    Example input: https:/example.com
    """
    args_schema: Type[ExtractWebsiteContextToolArgsSchema] = ExtractWebsiteContextToolArgsSchema

    def _run(self, url):
        loader = WebBaseLoader(url)
        return loader.load()
        


In [12]:
class SaveFileToolArgsSchema(BaseModel):
    doc: str


#OPENAI FUNCTION
class SaveFileTool(BaseTool):
    name = "SaveFileTool"
    description = """
    Save document found by ExtractWebsiteContextTool as a file(.txt).
    """
    args_schema: Type[SaveFileToolArgsSchema] = SaveFileToolArgsSchema

    def _run(self, doc):
        with open(f'files/research.txt', "w", encoding="utf-8") as f:
            f.write(doc)
            return f"{f.name} is created"
        

In [13]:
agent = initialize_agent(
    llm=llm, 
    model="gpt-4o-mini",
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        DuckDuckGoUrlSearchTool(), 
        WikipediaUrlSearchTool(),
        ExtractWebsiteContextTool(),
        SaveFileTool()
])

prompt = "Research about the XZ backdoor"

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `DuckDuckGoUrlSearchTool` with `{'query': 'XZ backdoor'}`


What Does the Backdoor Do? Malicious code added to xz Utils versions 5.6.0 and 5.6.1 modified the way the software functions. The backdoor manipulated sshd, the executable file used to make remote ... That operation matches the style of the XZ Utils backdoor far more than the cruder supply chain attacks of APT41 or Lazarus, by comparison. "It could very well be someone else," says Aitel. A backdoor was planted in the XZ utils package, a popular open-source compression utility for Linux/UNIX systems, by a threat actor named JiaT75. The backdoor exploited the SSH protocol to gain remote access and control over affected devices. The back door part comes into play with one of the main ways xz is used - SSH. SSH is an encrypted protocol between two machines where text commands can be exchanged, allowing a user to interact with a server. It's a very common utility in the Linux world

{'input': 'Research about the XZ backdoor',
 'output': 'I have compiled the research about the XZ backdoor into a text file. You can download it using the link below:\n\n[Download XZ Backdoor Research](sandbox:/files/research.txt) \n\nIf you need any further information or assistance, feel free to ask!'}